##  Creating the DB
This document is solely for testing localy, i am not sure if it can be applied to the current code but i will try my best to make it as compatible as possible

In [3]:
import sqlite3 
import os
import pandas as pd

### "Pre processing" the data 
it would be great to have the file path with it in each data frame
(this is nearly identical to what Anna is doing)

In [4]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\farah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
df = pd.read_excel('content\ground_truth\data.xlsx')
ground_truth_df = df[['Study_ID', 'Study', 'Allocation', 'Experimenter']]
ground_truth_df

,Study_ID,Study,Allocation,Experimenter
0,1,Chacko et al 2017,Random,Parent
1,1,Chacko et al 2017,Random,Parent
2,2,Coyne et al 2004,Non-random,Teacher
3,2,Coyne et al 2004,Non-random,Teacher
4,3,Purpura et al 2017,Random,Researcher
...,...,...,...,...
311,54,Hassinger-Das (2013),Random,Researcher
312,54,Hassinger-Das (2013),Random,Researcher
313,54,Hassinger-Das (2013),Random,Researcher
314,27,Crandell 2010,Random,Teacher


In [6]:
# extracting data from the csv files in the extracted directory

extracted_dir = 'content\extracted_split_2_shuffle_2'
extracted_dfs = []

if os.path.exists(extracted_dir):
    for filename in os.listdir(extracted_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(extracted_dir, filename)
            try:
                df = pd.read_csv(file_path)
                extracted_dfs.append(df)
                print(f"Loaded {filename}")
            except Exception as e:
                print(f"Failed to load {filename}: {e}")

if extracted_dfs:
    extracted_combined_df = pd.concat(extracted_dfs, ignore_index=True)
    print("Combined DataFrame created.")
else:
    print("No CSV files found in the directory.")

Loaded Bianco et al. - 2010 - Early Training in Oral Comprehension and Phonological Skills Results of a Three-Year Longitudinal S.pdf.csv
Loaded Blom-Hoffman et al. - 2007 - Instructing Parents to Use Dialogic Reading Strategies with Preschool Children Impact of a Video-Ba.pdf.csv
Loaded Carson - 2012 - Read with Me! Examining the Effects of a Community Volunteer Reading Program on Preschoolers' Litera.pdf.csv
Loaded Coyne et al. - 2004 - Teaching Vocabulary During Shared Storybook Readings An Examination of Differential Effects.pdf.csv
Loaded Coyne et al. - 2010 - Direct and Extended Vocabulary Instruction in Kindergarten Investigating Transfer Effects.pdf.csv
Loaded Crain-Thoreson and Dale - 1999 - Enhancing Linguistic Performance Parents and Teachers as Book Reading Partners for Children with La.pdf.csv
Loaded Crandell - 2010 - Information book read-alouds in Head Start preschools and the development of preschoolers' vocabular.pdf.csv
Loaded Herrell - 1989 - A child and an adult int

In [7]:
# directly taken from Anna's code
import re
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def format_studyName(study_name_string):
    """
    Cuts a string after the last four-digit number, assuming it represents the year.

    Args:
        study_name_string (str): The input string potentially containing a year.

    Returns:
        str: The string cut after the year, or the original string if no year is found.
    """
    # Get rid of all the points, -
    study_name_string = study_name_string.replace('.', '')
    study_name_string = study_name_string.replace(',', '')
    study_name_string = study_name_string.replace(' - ', ' ')
    study_name_string = study_name_string.replace(')', '')
    study_name_string = study_name_string.replace('(', '')
    study_name_string = study_name_string.replace('&', 'and')
    study_name_string = remove_accents(study_name_string)
    # Find all occurrences of four consecutive digits (potential years)
    year_matches = list(re.finditer(r'\b\d{4}\b', study_name_string))

    if year_matches:
        # Get the last match
        last_year_match = year_matches[-1]
        # Get the end index of the last year match
        end_of_year_index = last_year_match.end()
        # Slice the string up to the end of the year
        cut_string = study_name_string[:end_of_year_index]
        return cut_string.strip() # Use strip to remove trailing whitespace
    else:
        # If no four-digit number is found, return the original string
        return study_name_string.strip()

In [8]:
extracted_combined_df['Study'] = extracted_combined_df['Study'].apply(format_studyName)
extracted_combined_df

,Allocation,Experimenter,Study
0,Random,Parent,Bianco et al 2010
1,Random,Parent,Blom-Hoffman et al 2007
2,Random,Parent,Carson 2012
3,Random,Parent,Coyne et al 2004
4,Random,Parent,Coyne et al 2010
5,Random,Parent,Crain-Thoreson and Dale 1999
6,Random,Parent,Crandell 2010
7,Random,Parent,Herrell 1989
8,Random,Parent,Korat and Shamir 2007
9,Random,Parent,Korat et al 2013


### Loading the dataBase

In [9]:
def creatingDB(gt, extracted, table_nameGt, table_name_extracted):
  '''creates a SQLite database with the same structure as the given DataFrame and inserts the data into it.
  Args:
      gt (DataFrame): The ground truth DataFrame.
      extracted (DataFrame): The extracted DataFrame.
      table_nameGt (str): The name of the table for ground truth data.
      table_name_extracted (str): The name of the table for extracted data.
  '''
  
  # Connect to database
  conn = sqlite3.connect('content/database.db')

  # Dropping the tables if they exist
  conn.execute(f"DROP TABLE IF EXISTS {table_nameGt}")
  conn.execute(f"DROP TABLE IF EXISTS {table_name_extracted}")

  # Automatically create table with same name and structure
  gt.to_sql(table_nameGt, conn, if_exists='append', index=False)
  extracted.to_sql(table_name_extracted, conn, if_exists='append', index=False)

  print("Tables created with the same structure as the CSV and data inserted.")
  
  # Done
  conn.commit()
  conn.close()

In [10]:
gt_table = 'Ground_truth'
extracted_table = 'Extracted'
creatingDB(ground_truth_df, extracted_combined_df, gt_table, extracted_table)

Tables created with the same structure as the CSV and data inserted.


## Calculating the mesures
Auxilary functions used to calculate the mesures

In [11]:
def calculateAccuracy(TP, FP, FN):
    '''calculates the accuracy of a model based on true positives, true negatives, false positives, and false negatives.'''
    return TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

def calculatePrecision(TP, FP):
    '''calculates the precision of a model based on true positives and false positives.'''
    return TP / (TP + FP) if (TP + FP) > 0 else 0

def calculateRecall(TP, FN):
    '''calculates the recall of a model based on true positives and false negatives.'''
    return TP / (TP + FN) if (TP + FN) > 0 else 0

def calculateF1Score(precision, recall):
    '''calculates the F1 score based on precision and recall.'''
    return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

def calculateMetrics(TP, FP, FN):
    '''calculates various metrics based on true positives, false positives, and false negatives.'''
    # --- Metrics ---
    accuracy = calculateAccuracy(TP, FP, FN) 
    recall = calculateRecall(TP, FN)
    precision = calculatePrecision(TP, FP)
    f1 = calculateF1Score(precision, recall)

    return {
        'Accuracy': round(accuracy, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'F1': round(f1, 4)
    }


## SQL queries

In [12]:
def printAllTables():
    '''affiche toutes les tables de la base de données.'''

    conn = sqlite3.connect('content/database.db')
    cursor = conn.cursor()

    # testing if the data is inserted correctly
    cursor.execute(f"SELECT * FROM {gt_table} LIMIT 15;")
    print("Ground Truth Table:")
    for row in cursor.fetchall():
        print(row)

    cursor.execute(f"SELECT * FROM {extracted_table} ;")
    print("\nExtracted Table:")
    for row in cursor.fetchall():
        print(row)

    conn.commit()
    conn.close()

def countElements():
    '''counts the number of elements in each table and prints the result.'''

    conn = sqlite3.connect('content/database.db')
    cursor = conn.cursor()

    # testing if the data is inserted correctly
    cursor.execute(f"SELECT COUNT(*) FROM {gt_table};")
    print("Ground Truth Table count:")
    print(cursor.fetchall()[0])
        

    cursor.execute(f"SELECT COUNT(*) FROM {extracted_table} ;")
    print("\nExtracted Table count:")
    print(cursor.fetchall()[0])

    cursor.execute(f"""
        SELECT COUNT(ext.Study)
        FROM {gt_table} AS gt
                JOIN {extracted_table} AS ext ON gt.Study = ext.Study
        """)
    print("Total matched rows:", cursor.fetchone()[0])


    conn.commit()
    conn.close()

countElements()
printAllTables()

Ground Truth Table count:
(316,)

Extracted Table count:
(17,)
Total matched rows: 31
Ground Truth Table:
(1, 'Chacko et al 2017', 'Random', 'Parent')
(1, 'Chacko et al 2017', 'Random', 'Parent')
(2, 'Coyne et al 2004', 'Non-random', 'Teacher')
(2, 'Coyne et al 2004', 'Non-random', 'Teacher')
(3, 'Purpura et al 2017', 'Random', 'Researcher')
(3, 'Purpura et al 2017', 'Random', 'Researcher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(4, 'Vaknin-Nusbau & Nevo 2017', 'Random', 'Teacher')
(5, 'Wing-Yin Chow & McBride-Chang 2003', 'Random', 'Parent')
(5, 'Wing-Yin Chow & McBride-Chang 2003', 'Random', 'Parent')
(5, 'Wing-Yin Chow & McBride-Chang 2003', 'Random', 'Parent')

Extracted Table:
('Random', 'Parent', 'Bianco et al 2010')
('Random', 'Parent', 'Blom-Hoffman e

## Mesures

### Mesuring For binary values

In [13]:
def truePositives(conn, gt, extracted, column = 'Allocation', positiveValue = 'Random'):
    '''calculates the number of true positives based on ground truth and extracted data.
        where the ground thruth and extracted data both have the value 'Random'

        Args:
            conn (str): The path to the SQLite database.
            gt (str): The name of the ground truth table.
            extracted (str): The name of the extracted data table.
            column (str): The column to check for the positive value. Default is 'Allocation'.
            positiveValue (str): The value to check for in the specified column. Default is 'Random'.
        Returns:
            int: The count of true positives.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM {gt} AS gt 
            JOIN {extracted} AS ext ON gt.Study = ext.Study 
        WHERE gt.{column} = '{positiveValue}' AND ext.{column} = '{positiveValue}'
    """)
    result = cursor.fetchone()
    TP_count = result[0]
    
    print(f"Number of True Positives for '{column}': {TP_count}")

    conn.commit()
    conn.close()
    return TP_count

def falsePositives(conn, gt, extracted, column = 'Allocation', positiveValue = 'Random' ):
    '''calculates the number of true positives based on ground truth and extracted data.
        where gt = Random (1) and extracted (model output) = Non-random (0)
        Args:
            conn (str): The path to the SQLite database.
            gt (str): The name of the ground truth table.
            extracted (str): The name of the extracted data table.
            column (str): The column to check for the positive value. Default is 'Allocation'.
            positiveValue (str): The value to check for in the specified column. Default is 'Random'.
        Returns:
            int: The count of false positives.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    
    cursor.execute(f"""
        SELECT COUNT(*) FROM {gt} AS gt
        JOIN {extracted} AS ext ON gt.Study = ext.Study
        WHERE gt.{column} = '{positiveValue}' AND ext.{column} != '{positiveValue}'
    """)
    result = cursor.fetchone()
    FP_count = result[0]
    
    print(f"Number of false Positives for '{column}': {FP_count}")

    conn.commit()
    conn.close()

    return FP_count

def falseNegatives(conn, gt, extracted, column = 'Allocation', positiveValue = 'Random'):
    '''calculates the number of true positives based on ground truth and extracted data.
        where gt = Non-random (0) and extracted (model output) = Random (1)
        Args:
            conn (str): The path to the SQLite database.
            gt (str): The name of the ground truth table.
            extracted (str): The name of the extracted data table.
            column (str): The column to check for the positive value. Default is 'Allocation'.
            positiveValue (str): The value to check for in the specified column. Default is 'Random'.
        Returns:
            int: The count of false negatives.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    
    cursor.execute(f"""
        SELECT COUNT(*) FROM {gt} AS gt
        JOIN {extracted} AS ext ON gt.Study = ext.Study
        WHERE gt.{column} != '{positiveValue}' AND ext.{column} = '{positiveValue}'
    """)
    result = cursor.fetchone()
    FN_count = result[0]
    
    print(f"Number of false Negatives for '{column}': {FN_count}")

    conn.commit()
    conn.close()

    return FN_count

def trueNegatives(conn, gt, extracted, column = 'Allocation', positiveValue = 'Random'):
    '''calculates the number of true negatives based on ground truth and extracted data.
        where gt = Non-random (0) and extracted (model output) = Non-random (0)
        Args:
            conn (str): The path to the SQLite database.
            gt (str): The name of the ground truth table.
            extracted (str): The name of the extracted data table.
            column (str): The column to check for the positive value. Default is 'Allocation'.
            positiveValue (str): The value to check for in the specified column. Default is 'Random'.
        Returns:
            int: The count of true negatives.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    
    cursor.execute(f"""
        SELECT COUNT(*) FROM {gt} AS gt
        JOIN {extracted} AS ext ON gt.Study = ext.Study
        WHERE gt.{column} != '{positiveValue}' AND ext.{column} != '{positiveValue}'
    """)
    result = cursor.fetchone()
    TN_count = result[0]
    
    print(f"Number of true Negatives for '{column}': {TN_count}")

    conn.commit()
    conn.close()

    return TN_count

In [14]:
#print("True Positives:")
TP = truePositives('content/database.db', gt_table, extracted_table, 'Allocation')

#print("false Positives:")
FP = falsePositives('content/database.db', gt_table, extracted_table, 'Allocation')

#print("false Negatives:")
FN = falseNegatives('content/database.db', gt_table, extracted_table, 'Allocation')

TN = trueNegatives('content/database.db', gt_table, extracted_table, 'Allocation')

print("Calculating metrics... : \n", calculateMetrics(TP, FP, FN))

Number of True Positives for 'Allocation': 12
Number of false Positives for 'Allocation': 0
Number of false Negatives for 'Allocation': 19
Number of true Negatives for 'Allocation': 0
Calculating metrics... : 
 {'Accuracy': 0.3871, 'Recall': 0.3871, 'Precision': 1.0, 'F1': 0.5581}


### Multi class values 
Exparimenter
Basically it's when the selected column can be more then 2 values (non binary), i think it can work with allocation (binary values) but i still did a separate case for it just in case

In [15]:
def truePositivesExperimenter(conn, gt, extracted, column = 'Experimenter', 
                              values = ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent']):
    
    '''calculates the number of true positives based on ground truth and extracted data.
        where the ground thruth and extracted data both have the value 'teacher', 'parent', 'researcher' or 'Combined - Teacher and Parent'
    Args:
        conn (str): The path to the SQLite database.
        gt (str): The name of the ground truth table.
        extracted (str): The name of the extracted data table.
        column (str): The column to check for the positive value. Default is 'Experimenter'.
        values (list): The list of values to check for in the specified column. Default is ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent'].
    Returns:
        list: A list of counts of true positives for each value in the specified column.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    TP_count = []
    for value in values:
        cursor.execute(f"""
            SELECT COUNT(*)
            FROM {gt} AS gt 
                JOIN {extracted} AS ext ON gt.Study = ext.Study
            WHERE gt.{column} = '{value}' AND ext.{column} = '{value}'
        """)
        result = cursor.fetchone()
        for row in result:
            print(f"True Positives for {value}: {row}")
            TP_count.append(row)
    
    conn.commit()
    conn.close()
    return TP_count

def falsePositivesExperimenter(conn, gt, extracted, column = 'Experimenter', 
                              values = ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent']):
    
    '''calculates the number of False positives based on ground truth and extracted data.
        where the ground thruth and extracted data both have the value 'teacher', 'parent', 'researcher' or 'Combined - Teacher and Parent'
    Args:
        conn (str): The path to the SQLite database.
        gt (str): The name of the ground truth table.
        extracted (str): The name of the extracted data table.
        column (str): The column to check for the positive value. Default is 'Experimenter
        values (list): The list of values to check for in the specified column. Default is ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent'].
    Returns:
        list: A list of counts of false positives for each value in the specified column.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    TP_count = []
    for value in values:
        cursor.execute(f"""
            SELECT COUNT(*)
            FROM {gt} AS gt 
                JOIN {extracted} AS ext ON gt.Study = ext.Study
            WHERE gt.{column} != '{value}' AND ext.{column} = '{value}'
        """)
        result = cursor.fetchone()
        for row in result:
            print(f"False Positives for {value}: {row}")
            TP_count.append(row)
    
    conn.commit()
    conn.close()
    return TP_count

def falseNegativesExperimenter(conn, gt, extracted, column = 'Experimenter', 
                              values = ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent']):
    
    '''calculates the number of False nergatives based on ground truth and extracted data.
        where the ground thruth and extracted data both have the value 'teacher', 'parent', 'researcher' or 'Combined - Teacher and Parent'
    Args:
        conn (str): The path to the SQLite database.
        gt (str): The name of the ground truth table.
        extracted (str): The name of the extracted data table.
        column (str): The column to check for the positive value. Default is 'Experimenter'.
        values (list): The list of values to check for in the specified column. Default is ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent'].
    Returns:
        list: A list of counts of false negatives for each value in the specified column.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    FN_count = []
    for value in values:
        cursor.execute(f"""
            SELECT COUNT(*)
            FROM {gt} AS gt 
                JOIN {extracted} AS ext ON gt.Study = ext.Study
            WHERE gt.{column} = '{value}' AND ext.{column} != '{value}'
        """)
        result = cursor.fetchone()
        for row in result:
            print(f"False negatives for {value}: {row}")
            FN_count.append(row)
    
    conn.commit()
    conn.close()
    return FN_count

def trueNegativesExperimenter(conn, gt, extracted, column = 'Experimenter', 
                              values = ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent']):
    
    '''calculates the number of True negatives based on ground truth and extracted data.
        where the ground thruth and extracted data both have the value 'teacher', 'parent', 'researcher' or 'Combined - Teacher and Parent'
    Args:
        conn (str): The path to the SQLite database.
        gt (str): The name of the ground truth table.
        extracted (str): The name of the extracted data table.
        column (str): The column to check for the positive value. Default is 'Experimenter'.
        values (list): The list of values to check for in the specified column. Default is ['Teacher','Parent', 'Researcher', 'Combined - Teacher and Parent'].
    Returns:
        list: A list of counts of true negatives for each value in the specified column.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    TN_count = []
    for value in values:
        cursor.execute(f"""
            SELECT COUNT(*)
            FROM {gt} AS gt 
                JOIN {extracted} AS ext ON gt.Study = ext.Study
            WHERE gt.{column} != '{value}' AND ext.{column} != '{value}'
        """)
        result = cursor.fetchone()
        for row in result:
            print(f"True Negatives for {value}: {row}")
            TN_count.append(row)
    
    conn.commit()
    conn.close()
    return TN_count

In [16]:
##### use a function instead to group all this ######

TPE = truePositivesExperimenter('content/database.db', gt_table, extracted_table, 'Experimenter') 
TPE_sum = sum(TPE)
print("Sum of TPE:", TPE_sum)

FPE = falsePositivesExperimenter('content/database.db', gt_table, extracted_table, 'Experimenter')
FPE_sum = sum(FPE)
print("Sum of FPE:", FPE_sum)

FNE = falseNegativesExperimenter('content/database.db', gt_table, extracted_table, 'Experimenter')
FNE_sum = sum(FNE)
print("Sum of FNE:", FNE_sum)

calculateMetrics(TPE_sum, FPE_sum, FNE_sum)

True Positives for Teacher: 0
True Positives for Parent: 6
True Positives for Researcher: 0
True Positives for Combined - Teacher and Parent: 0
Sum of TPE: 6
False Positives for Teacher: 0
False Positives for Parent: 24
False Positives for Researcher: 0
False Positives for Combined - Teacher and Parent: 0
Sum of FPE: 24
False negatives for Teacher: 22
False negatives for Parent: 0
False negatives for Researcher: 2
False negatives for Combined - Teacher and Parent: 0
Sum of FNE: 24


{'Accuracy': 0.1111, 'Recall': 0.2, 'Precision': 0.2, 'F1': 0.2}

In [17]:
#provided by Anna 
def accuracy_check(col_name, df_extracted, test_table):
  allocation_match = False
  experimenter_match = False

  # Check if both dataframes have the expected columns and rows
  if col_name in df_extracted.columns and \
    not df_extracted.empty and not test_table.empty:

      extracted_allocation = df_extracted[col_name].iloc[0]

      ground_truth_allocation = test_table[col_name].iloc[0]

      # Simple case-insensitive comparison
      if str(extracted_allocation).lower() == str(ground_truth_allocation).lower():
          allocation_match = True
          print(f'{col_name}: Match')
      else:
          print(f"{col_name}: Mismatch (Extracted: '{extracted_allocation}', Ground Truth: '{ground_truth_allocation}')")
  else:
    print("Cannot perform accuracy check: Extracted or ground truth data is missing or malformed.")
  print("--------------------")

## Useful queries 
i'm just testing things out i don't really know what they want

In [18]:
def idkWhatToFind(conn, gt, ext):
    '''. Distribution of Experimenter Types by Allocation
        Sees how Experimenter types are distributed for each Allocation value in ground truth:
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    cursor.execute(f"""
                    SELECT Allocation, Experimenter, COUNT(*) AS Count
                    FROM {gt}
                    GROUP BY Allocation, Experimenter
                    ORDER BY Allocation, Count DESC; """)
    test_table = cursor.fetchall()
    print("Distribution of Experimenter Types by Allocation")
    print("in the ground truth table:")
    for row in test_table:
        print(row)
    
    cursor.execute(f"""
                    SELECT Allocation, Experimenter, COUNT(*) AS Count
                    FROM {ext}
                    GROUP BY Allocation, Experimenter
                    ORDER BY Allocation, Count DESC; """)
    test_table = cursor.fetchall()
    print("in the extracted table:")
    for row in test_table:
        print(row)
    
    conn.commit()
    conn.close()

In [19]:
idkWhatToFind('content/database.db', gt_table, extracted_table)

Distribution of Experimenter Types by Allocation
in the ground truth table:
('Non-random', 'Teacher', 30)
('Non-random', 'Researcher', 11)
('Non-random', None, 6)
('Non-random', 'Parent', 2)
('Random', 'Parent', 111)
('Random', 'Teacher', 81)
('Random', 'Researcher', 40)
('Random', 'Combined - Teacher and Parent', 24)
('Random', None, 11)
in the extracted table:
('Random', 'Parent', 17)


In [20]:
def Studies_with_missing_data(conn, gt, ext):
    '''Find studies present in both tables but with missing (NULL or empty) values in key columns:
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()
    
    cursor.execute(f"""
                    SELECT {gt}.Study, {gt}.Allocation AS GT_Allocation, {ext}.Allocation AS EXT_Allocation,
                    {gt}.Experimenter AS GT_Experimenter, {ext}.Experimenter AS EXT_Experimenter
                    FROM {gt}
                    JOIN {ext} ON {gt}.Study = {ext}.Study
                    WHERE {ext}.Allocation IS NULL
                        OR {ext}.Experimenter IS NULL
                        OR {ext}.Allocation = ''
                        OR {ext}.Experimenter = ''; """)
    test_table = cursor.fetchall()
    if not test_table:
        print("No studies with missing data found.")

    else : 
        print("Studies with Missing Data : ")
        for row in test_table:
            print(row)
    
    conn.commit()
    conn.close()

In [21]:
Studies_with_missing_data('content/database.db', gt_table, extracted_table)

No studies with missing data found.


In [ ]:
######## this is not what i wanted to do ########
def DistributionOfCathegories(conn, gt, ext, column = 'Allocation', cathegory = ['Random', 'Non-random']):
    '''Distribution of Categories in Ground Truth and Extracted Data
        Compares the distribution of categories in the ground truth and extracted data.
    '''
    conn = sqlite3.connect(conn)
    cursor = conn.cursor()

    for cat in cathegory:
        cursor.execute(f"""
                        SELECT COUNT(*) FROM {gt} GROUP BY {cat};
                        """)
        gt_count = cursor.fetchone()[0] 
        
        cursor.execute(f"""
                        SELECT COUNT(*) FROM {ext} GROUP BY {cat};
                        """)
        ext_count = cursor.fetchone()[0]
        
        print(f"Category '{cat}': Ground Truth Count: {gt_count}, Extracted Count: {ext_count}")
    
    conn.commit()
    conn.close()

def plotDistributionByCathegory(gt_count, ext_count) :
    '''bar plot of the distribution of categories in the ground truth and extracted data.'''

    ##### place holder for the plot #####


In [ ]:
DistributionOfCathegories('content/database.db', gt_table, extracted_table, 'Allocation')  
DistributionOfCathegories('content/database.db', gt_table, extracted_table, 'Experimenter')

## To do :
- test the other written queries
- test with the multiple shots prompting (it's only done with the python code i should've started here)